### Summarization with BART 

### We will use a pretrained Transformers Flax BART model for summarization `FlaxBartForConditionalGeneration.from_pretrained` 

## **1. Set up/Installation**

In [1]:
!nvcc --version
!python --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Python 3.7.15


In [2]:
!pip install --upgrade jax 
!pip install flax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Verify if Jax/Flax were installed correctly

In [3]:
!transformers-cli env

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
2022-11-09 23:41:14.818104: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.

Copy-and-paste the text below in your GitHub issue and FILL OUT the two last points.

- `transformers` version: 4.25.0.dev0
- Platform: Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
- Python version: 3.7.15
- Huggingface_hub version: 0.10.1
- PyTorch version (GPU?): 1.12.1+cu113 (True)
- Tensorflow version (GPU?): 2.9.2 (True)
- Flax version (CPU?/GPU?/TPU?): 0.6.1 (gpu)
- Jax version: 0.3.24
- JaxLib version: 0.3.22
- Using GPU in script?: <fill in>
- Using distributed or parallel set-up in script?: <fill in>



### Clone the Hugging Face transformer library from Github.

In [4]:
!git clone https://github.com/huggingface/transformers && cd transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [5]:
!pip install ./transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.25.0.dev0-py3-none-any.whl size=5602445 sha256=ad81ef0114cfa28055e432d212dc3cd9e818ce66f2fc35fe9279d3003cf73e10
  Stored in directory: /tmp/pip-ephem-wheel-cache-4qhcxpae/wheels/49/62/f4/6730819eed4e6468662b1519bf3bf46419b2335990c77f8767
Successfully built transformers
  Attempting uninstall: t

### Check if we are running on GPU

In [6]:
import jax
import jax.numpy as np

key = jax.random.PRNGKey(0)

print('JAX is running on', jax.lib.xla_bridge.get_backend().platform)

JAX is running on gpu


## **2. Run Flax BART summarization example**

### Generate 2 summaries of different lengths from an sample article

In [20]:
from transformers import BartTokenizer, FlaxBartForConditionalGeneration
from IPython.display import display, Markdown

model = FlaxBartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

ARTICLE_TO_SUMMARIZE = """
The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey 
building, and the tallest structure in Paris. Its base is square, measuring 125
metres (410 ft) on each side. During its construction, the Eiffel Tower 
surpassed the Washington Monument to become the tallest man-made structure in 
the world, a title it held for 41 years until the Chrysler Building in New York 
City was finished in 1930. It was the first structure to reach a height of 300 
metres. Due to the addition of a broadcasting aerial at the top of the tower in 
1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). 
Excluding transmitters, the Eiffel Tower is the second tallest free-standing 
structure in France after the Millau Viaduct.
"""
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors="np")

# Generate Summary of different lengths (56 words by default)
# With at least 30 words
summary_ids_30 = model.generate(inputs["input_ids"], min_length=30).sequences

# With at least 50 words
summary_ids_50 = model.generate(inputs["input_ids"], min_length=50).sequences

summary_txt_30 = tokenizer.batch_decode(summary_ids_30, skip_special_tokens=True, clean_up_tokenization_spaces=False)
summary_txt_50 = tokenizer.batch_decode(summary_ids_50, skip_special_tokens=True, clean_up_tokenization_spaces=False)

display(Markdown('> **Summary (30 words limit): ** '+summary_txt_30[0]))
display(Markdown('> **Summary (50 words limit): ** '+summary_txt_50[0]))


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/generation/flax_utils.py:352: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 142 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/transformers/generation/flax_utils.py:352: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 142 (

> **Summary (30 words limit): ** The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building. Its base is square, measuring 125 metres (410 ft) on each side.

> **Summary (50 words limit): ** The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building. Its base is square, measuring 125 metres (410 ft) on each side. It is the second tallest free-standing structure in France after the Millau Viaduct.